<a href="https://colab.research.google.com/github/aulia-adil/craiglist-sales/blob/dewa/Proyek_Akhir_KASDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rumusan masalah

1. **Dataset Craiglist Sales (DCS) ini apa sih?** DCS adalah dataset terkait mobil second yang dijual oleh Perusahaan Craiglist. Tentunya dataset tersebut juga disertai dengan data-data relevan terkait penjualan mobil second tersebut.
1. **(Optional) Mengapa DCS dibuat?**
1. **(Optional) Bagaimana DCS dibuat?**
1. **(Optional) Apakah DCS akurat?**
1. **Apa karakteristik DCS?**
1. **Apa definisi dari setiap atribut DCS?**
1. **Masalah-masalah apa saja yang dimiliki DCS?**
1. **Mengapa masalah-masalah tersebut muncul?**
1. **Apa solusi dari setiap masalah yang dimiliki DCS?**
1. **Apakah ada independent variable dari DCS? Jika iya apa saja?**
1. **Apakah ada dependent variable dari DCS? Jika iya apa saja dan bagaimana korelasi hubungannya?**
1. **Bagaimana cara visualisasi dari setiap atribut yang ada pada DCS?**
1. **Apa makna dari setiap distribusi yang dimiliki atribut-atribut DCS? Apakah ada anomali di dalamnya?**
1. **(Optional) Apakah DCS dapat dikaitkan dengan dataset lain?**
1. **Apa yang bisa kita dapatkan dari DCS ini?**

## Import Module dan Dataset

In [1]:
# Library Adil nanti dibersihin aja
# import library
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as scp
import statsmodels.api as sm
import sys
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as scp
import statsmodels.api as sm
import sklearn.manifold as smn
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, Normalizer, StandardScaler, RobustScaler
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import scipy.stats as scp

In [3]:
def download_from_gdrive(file_id):
  downloaded = drive.CreateFile({'id':file_id})
  downloaded.FetchMetadata(fetch_all=True)
  downloaded.GetContentFile(downloaded.metadata['title'])

In [4]:
# import cell
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
# Colab harus menggunakan akun UI untuk bisa mendownload file ini
download_from_gdrive('1BpEsEOQyd9F7m4TC7oGfu6e0wKtHV7gj')

In [ ]:
!unzip 'archive.zip'

Archive:  archive.zip
replace vehicles.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# Read dataset
data = pd.read_csv("vehicles.csv")

## Exploratory Data Analysis dan Preprocessing Raw Data

Pertanyaan EDA Raw:

1. Apakah setiap atribut numerik bertipe numerik?
1. Apakah setiap atribut kategorik bertipe kategorik?
1. Berapa banyak persentase missing values setiap atribut?
1. Apakah ada row duplikat?
1. Apakah ada outlier?

In [ ]:
#@title Fungsi
def see_dataset_dimension(data):
  print("Dimension")
  print("Row:", data.shape[0])
  print("Column:", data.shape[1])

def see_numerical_attribute(data):
  num_attr = data.select_dtypes(include=[np.number])
  print("Numerical Attribute")
  i = 1
  for column in num_attr.columns:
    print(str(i) + ".", column)
    i += 1

def see_categorical_attribute(data):
  cat_attr = data.select_dtypes(exclude=[np.number])
  print("Categorical Attribute")
  i = 1
  for column in cat_attr.columns:
    print(str(i) + ".", column)
    i += 1

def count_outlier_column(data, column):
  q1 = data.quantile(q=0.25)[column]
  q3 = data.quantile(q=0.75)[column]
  iqr = q3-q1
  upper_bound = q3 + iqr * 1.5
  lower_bound = q1 - iqr * 1.5
  upper_outliers = data.loc[data[column] > upper_bound, [column]]
  lower_outliers = data.loc[data[column] < lower_bound, [column]]
  return len(upper_outliers) + len(lower_outliers)

def numeric_descriptive_statistic(df):
  df = df.select_dtypes(include=[np.number])
  desc_stat = df.describe()
  columns = df.columns
  mode_list = []
  variance_list = []
  range_list = []
  skewness_list = []
  outlier_list = []
  for column in columns:
    outlier_list.append(count_outlier_column(df, column))
    skewness_list.append(df[column].skew())
    range = desc_stat[column]["max"]-desc_stat[column]["min"]
    range_list.append(range)
    variance = desc_stat[column]["std"] ** 2
    variance_list.append(variance)
    mode_list.append(scp.mode(df[column])[0][0])
  desc_stat.loc["mode"] = mode_list
  desc_stat.loc["variance"] = variance_list
  desc_stat.loc["range"] = range_list
  desc_stat.loc["skewness"] = skewness_list
  desc_stat.loc["total outlier"] = outlier_list
  return desc_stat

def numeric_cleaning_fillna_mean(df):
  df_res = df.select_dtypes(include=[np.number])
  columns = df_res.columns
  for column in columns:
    mean = df_res[column].mean()
    df_res[column].fillna(value=mean, inplace=True)
  return df_res

# Cek missing value
def cek_null(df):
    col_na = df.isnull().sum().sort_values(ascending=False)
    percent = col_na / len(df)
    
    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_data[missing_data['Total'] > 0])

In [ ]:
print("Dimensi")
print("Baris:", data.shape[0])
print("Kolom:", data.shape[1])

print()
num_attr = data.select_dtypes(include=[np.number])
print("Atribut numerik")
i = 1
for column in num_attr.columns:
  print(str(i) + ".", column)
  i += 1

print("\nAtribut kategorik")
cat_attr = data.select_dtypes(exclude=[np.number])
i = 1
for column in cat_attr.columns:
  print(str(i) + ".", column)
  i += 1

print("\nDescriptive Statistic Numerical Attribute")
num_desc_stat = numeric_descriptive_statistic(data)
display(num_desc_stat)

print("\nDescriptive Statistic Categorical Attribute")
display(cat_attr.describe())

print("\nLihat persentase missing values")
display(cek_null(data))

print("\nHitung duplikasi data")
no_dup = data.drop_duplicates(inplace=False)
print("Jumlah duplikasi data:", len(data) - len(no_dup))

print("\nHitung outlier")
display(pd.DataFrame(num_desc_stat.loc['total outlier']))

In [ ]:
print("Atribut drive dan cylinders terlihat numerik")
print()
print("Atribut drive")
display(cat_attr['drive'].dropna())
print()
print("Atribute cylinders")
display(cat_attr['cylinders'].dropna())

#### Identifikasi masalah dataset

1. Atribut `cylinders` disebut atribut kategorik tetapi sebenarnya mirip dengan atribut numerik.
1. Atribut `county` semuanya null values.
1. Atribut `size` hingga `paint color` memiliki banyak missing values.
1. Banyak outlier pada atribut numerik, hal ini perlu di visualisasi persebarannya.

In [ ]:
cleaned_data = data.copy()
# Drop kolom county
cleaned_data.drop(columns = ['county'], inplace = True)

#### Visualisasi banyak iklan yang diposting setiap state

In [ ]:
print("State yang paling banyak memposting iklan adalah CA")
plt.figure(figsize = (20, 10))
sns.countplot(x = 'state', data = data)
plt.xlabel("state")
plt.xticks(rotation = "vertical")
plt.show()

#### Visualisasi jumlah nilai odometer untuk setiap state

In [ ]:
# (Hint di soal disuruh pake year sama odometer, tapi menurutku odometer cukup)
state_odo = cleaned_data.loc[:, ['state', 'odometer']]
state_odo = state_odo.groupby('state', as_index = False).sum()

plt.figure(figsize = (20, 10))
sns.barplot(x = 'state', y = 'odometer', data = state_odo)
plt.xlabel("state")
plt.ylabel("sum of odometer values")
plt.xticks(rotation = "vertical")
plt.show()
print("State yang paling sering menggunakan mobil adalah CA")

#### Jenis kendaraan paling populer untuk setiap state

##### Berdasarkan type

In [ ]:
state_type = cleaned_data.loc[:, ['state', 'type']]
grid = sns.catplot(x = 'type', col = 'state', col_wrap = 8, data = state_type, kind = 'count', height = 2.5)
grid.set_xticklabels(rotation = "vertical")
plt.show()

# Test Model

In [ ]:
def cylinder_encode(x):
  if x not in [np.nan, 'other']:
    return int(x.split()[0])
  else:
    return x

new_data = cleaned_data.copy()
cylinders = new_data.loc[~new_data['cylinders'].isin([np.nan, 'other']), 'cylinders']
cylinders = cylinders.apply(cylinder_encode)

cyl_mode = cylinders.quantile(0.5)
new_data['cylinders'] = new_data['cylinders'].apply(cylinder_encode)
new_data['cylinders'].replace(to_replace = [np.nan, 'other'], value = [cyl_mode, cyl_mode], inplace = True)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

X = new_data.loc[:, ['cylinders', 'odometer']].fillna(0)
y = new_data.loc[:, ['price']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 18)

#reg = MLPRegressor()
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

print("R2 square:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))